# 1. Prueba de módulo "webscraping"

**scrap_discursos:** Scrap de discursos desde una web paginada con links individuales por discurso.

Parámetros:
- base_url (str): URL de inicio.
- xpaths (dict): Diccionario con claves: 'link_items', 'boton_siguiente', 'titulo', 'fecha', 'contenido'.
- espera (int): Tiempo de espera entre interacciones.
- paginas (int): Número máximo de páginas a recorrer.
- articulos_maximos (int): Límite de discursos a scrapear.
- headless (bool): Si True, navegador sin interfaz.
- verbose (bool): Si True, muestra estado.
- output_path (str): Ruta opcional para guardar CSV.
- mostrar_tiempo (bool): Si True, muestra tiempo de procesamiento.

Retorna:
- DataFrame con columnas: 'url', 'titulo', 'fecha', 'contenido', 'codigo'

In [1]:
from modulos.webscraping import xpaths_casarosada, scrap_discursos
import modulos.paths as paths

df = scrap_discursos(
    base_url="https://www.casarosada.gob.ar/informacion/discursos",
    xpaths=xpaths_casarosada,
    espera=3,
    paginas=5,
    articulos_maximos=1,
    headless=True,
    verbose=True,
    output_path=paths.discursos,
    mostrar_tiempo=True
)

df.head()

C:\ProgramData\anaconda3\envs\ag_env2\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


🌐 Página 1
➕ 40 nuevos links

🔗 Total de links únicos obtenidos: 1

📄 (1/1) Procesando: https://www.casarosada.gob.ar/informacion/discursos/51058-palabras-del-presidente-de-la-nacion-javier-milei-ante-el-consejo-interamericano-de-comercio-y-produccion-cicyp-2025
📝 Palabras del Presidente de la Nación, Javier Milei, ante el Consejo Interamericano de Comercio y Producción (CICyP) 2025 (Jueves 28 de agosto de 2025) - 7908 palabras
✅ Archivo guardado: C:\PROYECTOS\EmoParse\data\A1. discursos.csv
⏱ Tiempo total de procesamiento: 65.51 s


,url,titulo,fecha,contenido,codigo,INDEX
0,https://www.casarosada.gob.ar/informacion/disc...,"Palabras del Presidente de la Nación, Javier M...",Jueves 28 de agosto de 2025,"Palabras del Presidente de la Nación, Javier M...",DISCURSO_001,0


# 2. Prueba de módulo "preprocesamiento"

## 2.1. Segmentación en recortes

**generar_recortes:** Convierte una base de discursos en una base de recortes frase a frase.

Parámetros:
- df_discursos (DataFrame): Debe tener columnas 'titulo' y 'contenido'.
- agregar_codigo (bool): Si True, genera columna 'codigo' con formato DISCURSO_001_FR_001...
- prefijo_codigo (str): Prefijo para los códigos asignados.
- guardar (bool): Si True, guarda la base generada como CSV.
- output_path (str): Ruta del archivo CSV de salida (requerido si guardar=True).
- mostrar_tiempo (bool): Si True, muestra tiempo de procesamiento.

Retorna:
- DataFrame con columnas: 'codigo', 'recorte_id', 'posicion', 'frase'.

In [2]:
import pandas as pd
import modulos.paths as paths
from modulos.preprocesamiento import generar_recortes

# Cargar discursos scrapeados
df_discursos = pd.read_csv(paths.discursos, encoding="utf-8-sig")

# Generar recortes y guardar
df_recortes = generar_recortes(df_discursos, guardar=True, output_path=paths.recortes, mostrar_tiempo=True)

✅ Archivo guardado: C:\PROYECTOS\EmoParse\data\B1. recortes.csv
🧾 La base tiene 369 observaciones (frases).
⏱ Tiempo de generar_recortes: 0.02 s


## 2.2. Filtrado de discursos por cantidad de frases (mínimo: 24, para series temporales)

**filtrar_discursos:** Filtra los códigos que tienen al menos `umbral` frases en `df_recortes`, y opcionalmente guarda los resultados si se indican los paths.

Parámetros:
- df: DataFrame original con los textos completos.
- df_recortes: DataFrame con las frases (recortes) y la columna 'codigo'.
- umbral: Cantidad mínima de frases necesarias para conservar un código.
- guardar: Si True, guarda los DataFrames filtrados y la lista de códigos eliminados.
- path_discursos: Ruta para guardar el CSV de discursos filtrados.
- path_recortes: Ruta para guardar el CSV de recortes filtrados.
- path_codigos_eliminados: Ruta para guardar el TXT con los códigos eliminados.
- mostrar_tiempo (bool): Si True, muestra tiempo de procesamiento.

Retorna:
- df_filtrado: DataFrame con discursos filtrados.
- df_recortes_filtrado: DataFrame de recortes correspondientes a los discursos filtrados.
- codigos_eliminados: lista de códigos de discursos eliminados.
- codigos_validos : lista de códigos de discursos válidos.
- conteo_frases: DataFrame con el conteo total de frases por código, antes del filtrado.

In [3]:
import pandas as pd
import modulos.paths as paths
from modulos.preprocesamiento import filtrar_discursos

df = pd.read_csv(paths.discursos, encoding="utf-8-sig")
df_recortes = pd.read_csv(paths.recortes, encoding="utf-8-sig")

df_filtrado, df_recortes_filtrado, codigos_eliminados, codigos_validos, conteo_frases = filtrar_discursos(
    df=df,
    df_recortes=df_recortes,
    umbral=24,
    guardar=True,
    path_discursos=paths.discursos_filtrado,
    path_recortes=paths.recortes_filtrado,
    path_codigos_eliminados=paths.codigos_eliminados,
    mostrar_tiempo=True
)

Cantidad de frases por código (primeras filas):
         codigo  cantidad_frases
0  DISCURSO_001              369

✅ Códigos con al menos 24 frases: 1
❌ Códigos eliminados (menos de 24 frases): 0

📄 Textos originales tras el filtro: 1
🧾 Frases tras el filtro: 369
✅ Archivo guardado: C:\PROYECTOS\EmoParse\data\A2. discursos_filtrado.csv
✅ Archivo guardado: C:\PROYECTOS\EmoParse\data\B2. recortes_filtrado.csv

💾 Códigos eliminados guardados en: C:\PROYECTOS\EmoParse\logs\codigos_eliminados.txt
⏱ Tiempo de filtrar_discursos: 0.01 s


## 2.3. Limpieza y preprocesamiento

**procesar_textos:** Procesa textos en una columna de un DataFrame aplicando limpieza y análisis lingüístico.

Parámetros:
- df (pd.DataFrame): DataFrame original.
- columna_texto (str): Nombre de la columna con texto.
- texto_limpio (bool): Si se debe incluir columna con texto limpio.
- tokens (bool): Si se deben incluir los tokens.
- lemmas (bool): Si se deben incluir los lemas.
- pos_tags (bool): Si se deben incluir etiquetas POS.
- dependencias (bool): Si se deben incluir dependencias sintácticas.
- entidades (bool): Si se deben incluir entidades nombradas.
- sujetos (bool): Si se debe marcar sujeto omitido por frase.
- guardar (bool): Si se debe guardar el resultado a CSV.
- path_salida (str): Ruta de salida (obligatoria si guardar=True).
- mostrar_tiempo (bool): Si True, muestra tiempo de procesamiento.

Retorna:
- pd.DataFrame: DataFrame con columnas nuevas según los flags seleccionados.

In [4]:
import pandas as pd
import modulos.paths as paths
from modulos.preprocesamiento import procesar_textos

df_filtrado = pd.read_csv(paths.discursos_filtrado, encoding="utf-8-sig")

df_resultado = procesar_textos(
    df=df_filtrado,
    columna_texto="contenido",
    texto_limpio=True,
    tokens=False,
    lemmas=False,
    pos_tags=False,
    dependencias=False,
    entidades=False,
    sujetos=False,
    guardar=True,
    path_salida=paths.discursos_preprocesado,
    mostrar_tiempo=True
)

✅ Archivo guardado: C:\PROYECTOS\EmoParse\data\A3. discursos_preprocesado.csv
⏱ Tiempo de procesar_textos: 1.17 s


In [5]:
df_recortes_filtrado = pd.read_csv(paths.recortes_filtrado, encoding="utf-8-sig")

df_resultado = procesar_textos(
    df=df_recortes_filtrado,
    columna_texto="frase",
    texto_limpio=True,
    tokens=True,
    lemmas=True,
    pos_tags=True,
    dependencias=True,
    entidades=True,
    sujetos=True,
    guardar=True,
    path_salida=paths.recortes_preprocesado,
    mostrar_tiempo=True
)

✅ Archivo guardado: C:\PROYECTOS\EmoParse\data\B3. recortes_preprocesado.csv
⏱ Tiempo de procesar_textos: 4.26 s


**Nota para optimización:** Ver de que no se dupliquen entidades y dependencias en el archivo

# 3. Prueba de módulo "resumen"

Este módulo permite generar resúmenes de discursos extensos utilizando un LLM, a través de un enfoque por bloques temáticos. La función principal aplicada es `resumir_dataframe`.

#### Lógica del procedimiento

- **Segmentación semántica:** el texto se divide en fragmentos según cambios temáticos detectados.
- **Resumen parcial:** cada fragmento se resume individualmente mediante un LLM.
- **Redacción global:** se produce un resumen final fluido e integrado a partir de los resúmenes parciales.

#### resumir_dataframe

Genera resúmenes utilizando un LLM para cada fila de un DataFrame que contenga la columna `texto_limpio`. Utiliza dos tipos de prompts: uno para fragmentos y otro para la redacción final del discurso completo.

#### Parámetros

- `df` (`pd.DataFrame`): DataFrame con columnas `'codigo'`, `'titulo'`, `'fecha'`, `'texto_limpio'`.
- `modelo_llm` (`Callable[[str], str]`): Función que toma un prompt (`str`) y devuelve la respuesta generada por el modelo LLM.
- `prompt_fragmento` (`str`): Prompt utilizado para resumir cada fragmento segmentado del discurso.
- `prompt_discurso` (`str`): Prompt utilizado para redactar el resumen global del discurso a partir de los resúmenes parciales.
- `umbral` (`float`): Umbral de sensibilidad para la segmentación temática. Valores más bajos generan más fragmentos.
- `guardar` (`bool`): Si es `True`, guarda el DataFrame resultante en un archivo `.csv`.
- `path_salida` (`str`): Ruta del archivo de salida si `guardar=True`.
- `mostrar_prompts` (`bool`): Si es `True`, imprime los prompts utilizados durante la ejecución (útil para debugging).
- `mostrar_tiempo` (`bool`): Si `True`, muestra tiempo de procesamiento.
- `max_chars_parciales` (`int`, opcional): Número máximo de caracteres que se permitirá en cada fragmento antes de enviarlo al LLM. Evita prompts excesivamente largos.
- `max_chars_final` (`int`, opcional): Número máximo de caracteres que se permitirá en el resumen final antes de enviarlo al LLM. Ayuda a controlar la extensión del texto generado.

#### Retorna

- `pd.DataFrame`: DataFrame con una nueva columna `'resumen'` que contiene el resumen generado por LLM para cada discurso.


In [6]:
import pandas as pd
import modulos.paths as paths
import modulos.prompts as prompts
from modulos.resumen import resumir_dataframe
from modulos.modelo import get_model_ollama_par

# Crear el modelo LLM
modelo_llm = get_model_ollama_par(modelo="gpt-oss:20b", temperature=0.0, output_format="text")

# Cargar los discursos preprocesados
df_preprocesado = pd.read_csv(paths.discursos_preprocesado, encoding="utf-8-sig")
print(f"📄 {len(df_preprocesado)} discursos cargados para resumir.")

# Resumir los discursos
df_con_resumenes = resumir_dataframe(
    df=df_preprocesado,
    modelo_llm=modelo_llm,
    prompt_fragmento=prompts.PROMPT_RESUMIR_FRAGMENTO,
    prompt_discurso=prompts.PROMPT_RESUMIR_DISCURSO,
    umbral=0.25,
    guardar=True,
    path_salida=paths.discursos_resumen,
    mostrar_prompts=False,
    mostrar_tiempo=True,
    max_chars_parciales=10000,
    max_chars_final=3000
)

C:\ProgramData\anaconda3\envs\ag_env2\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: distiluse-base-multilingual-cased-v1
C:\ProgramData\anaconda3\envs\ag_env2\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


📄 1 discursos cargados para resumir.


Generando resúmenes con LLM:   0%|                                                                                                                                                                                    | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
Generando resúmenes con LLM: 100%|██████


🟩 Resumen generado con éxito para código: DISCURSO_001
Resumen:
**Resumen del discurso de Javier Milei ante el Consejo Interamericano de Comercio y Producción (CICyP) – 28 de agosto 2025**

Javier Milei abrió su intervención agradeciendo la invitación de la presidenta Bettina Bulgheroni y reconociendo a las autoridades del CICyP. En los primeros minutos recordó un episodio de su campaña en el que un grupo violento arrojó piedras y difundió acusaciones absurdas sobre la venta de órganos y armas a menores. Con esta anécdota ilustró la resistencia de la casta política, “arraigada en el Estado”, que intenta frenar la transformación que el pueblo ha elegido.

El presidente reafirmó que su proyecto político se basa en la libertad y en la eliminación de privilegios. Anunció que el 7 de septiembre los bonaerenses acabarán con el “regimen de mentiras, violencia y corrupción” del kirchnerismo. Milei hizo un repaso de su programa: reducción de impuestos, eliminación del “impuesto país”, avance e

# 4. Prueba de módulos "metadatos" y "enunciacion"

Estos módulos permiten la identificación automática de tipo de discurso, enunciador, enunciatarios y lugar de enunciación a partir de un conjunto de discursos.
La tarea se organiza en dos funciones principales, correspondientes a cada módulo (metadatos.py y enunciacion.py) que pueden usarse por separado o en conjunto.

## 4.1. Procesar tipo de discurso y lugar

**procesar_metadatos_llm:** Identifica el tipo de discurso y el lugar de enunciación (ciudad, provincia, país). Utiliza un modelo de lenguaje (LLM) con prompts personalizados.

Parámetros:
- df (pd.DataFrame): DataFrame con columnas 'codigo', 'titulo', 'texto_limpio' y 'resumen'.
- modelo_llm (str): Modelo de lenguaje a utilizar (ej. "gpt-oss:20b").
- diccionario (dict): Diccionario conceptual con categorías y ejemplos, utilizado para orientar la detección de tipo de discurso (ej. diccionario_tipos_discurso).
- prompt_tipo (str): Prompt base para la detección del tipo de discurso. Debe incluir los placeholders "RESUMEN", "FRAGMENTOS" y "DICCIONARIO".
- prompt_lugar (str): Prompt base para la detección del lugar de enunciación. Debe incluir los placeholders "TITULO", "RESUMEN" y "FRAGMENTOS".
- guardar (bool, opcional): Si es True, guarda los resultados en un archivo .csv. Por defecto: True.
- output_path (str, opcional): Ruta al archivo donde se guardará el CSV. Obligatorio si guardar=True.
- mostrar_prompts (bool, opcional): Si es True, imprime los prompts construidos antes de enviarlos al modelo (útil para debugging). Por defecto: False.
- mostrar_tiempo (bool, opcional): Si es True, muestra el tiempo de procesamiento.

Retorna:
- pd.DataFrame: DataFrame original con columnas adicionales:
    - tipo_discurso, tipo_discurso_justificacion
    - lugar_ciudad, lugar_provincia, lugar_pais, lugar_justificacion

Requiere:
- Definición previa de los paths en el módulo modulos.paths.
- Disponibilidad de los prompts base en el módulo modulos.prompts.
- Diccionario conceptual diccionario_tipos_discurso importado desde modulos.tipos_discurso.

## 4.2. Procesar enunciador y enunciatarios

**procesar_enunciacion_llm:** Identifica el enunciador y los enunciatarios de un discurso, utilizando un modelo de lenguaje (LLM) con prompts específicos.

Parámetros:
- df (pd.DataFrame): DataFrame con columnas codigo, titulo, texto_limpio y resumen.
- modelo_llm (str): Modelo de lenguaje a utilizar.
- diccionario (dict): Diccionario conceptual con categorías y ejemplos, utilizado para orientar la identificación de roles enunciativos (ej. diccionario_tipos_discurso).
- prompt_enunciacion (str): Prompt base para la identificación del enunciador y los enunciatarios. Debe incluir los placeholders "RESUMEN", "FRAGMENTOS" y "DICCIONARIO".
- guardar (bool, opcional): Si es True, guarda los resultados en un archivo .csv. Por defecto: True.
- output_path (str, opcional): Ruta al archivo donde se guardará el CSV. Obligatorio si guardar=True.
- mostrar_prompts (bool, opcional): Si es True, imprime los prompts construidos antes de enviarlos al modelo.
- mostrar_tiempo (bool, opcional): Si es True, muestra el tiempo de procesamiento.

Retorna:
- pd.DataFrame: DataFrame original con columnas adicionales:
    - enunciador_actor, enunciador_justificacion
    - enunciatario_0_actor, enunciatario_0_tipo, enunciatario_0_justificacion
    - enunciatario_1_actor, ... (tantos como detecte el modelo).

Requiere:
- Definición previa de los paths en el módulo modulos.paths.
- Disponibilidad de los prompts base en el módulo modulos.prompts.
- Diccionario conceptual diccionario_tipos_discurso importado desde modulos.tipos_discurso.

### Instanciación del modelo LLM
Usamos output_format="text" en lugar de format="json" porque:

1. En la práctica, incluso GPT-OSS no siempre devuelve JSON válido.
2. StructuredOutputParser de LangChain permite parsear y validar la salida de manera confiable.
3. Combinando "text" + parser mantenemos robustez frente a pequeñas inconsistencias en la salida del LLM.
4. Esto simplifica el pipeline y evita errores de parseo que aparecían con format="json".

### Funciones de reprocesamiento
En ambos casos, se diseñaron funciones para reprocesar los errores registrados en la carpeta "errors".

In [7]:
# Cargar dataset con resúmenes
import pandas as pd
import modulos.paths as paths

df_discursos = pd.read_csv(paths.discursos_resumen, encoding="utf-8-sig")

# Procesar discursos con el LLM
import modulos.prompts as prompts
from modulos.metadatos import procesar_metadatos_llm
from modulos.tipos_discurso import diccionario_tipos_discurso
from modulos.modelo import get_model_ollama_par

modelo_llm = get_model_ollama_par(modelo="gpt-oss:20b", temperature=0.0, output_format="text")

# Procesar tipo de discurso y lugar
df_tipo_lugar = procesar_metadatos_llm(
    df=df_discursos,
    modelo_llm=modelo_llm,
    diccionario=diccionario_tipos_discurso,
    prompt_tipo=prompts.PROMPT_TIPO_DISCURSO,
    prompt_lugar=prompts.PROMPT_LUGAR,
    guardar=True,
    output_path=paths.discursos_metadatos,
    mostrar_tiempo=True,
    mostrar_prompts=False,
    path_errores_tipo=paths.errores_metadatos,
    path_errores_lugar=paths.errores_metadatos
)

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


⏱ Tiempo de procesar_tipo_lugar_llm: 29.02 s


In [8]:
# Reprocesar metadatos en caso de ser necesario

import pandas as pd
import modulos.paths as paths
from modulos.modelo import get_model_ollama_par
from modulos.reprocesamiento import reprocesar_errores_metadatos

path_errores = paths.errores_metadatos
path_salida = paths.discursos_metadatos

df_original = pd.read_csv(paths.discursos_metadatos)

modelo_llm = get_model_ollama_par(modelo="gpt-oss:20b", temperature=0.0, output_format="text")

# Ejecutar reprocesamiento
df_corregido = reprocesar_errores_metadatos(
    df_original=df_original,
    path_errores=path_errores,
    modelo_llm=modelo_llm,
    intento=1,
    mostrar_prompts=False,
    path_salida=path_salida
)

# Nota: Esta función debe volver a ser probada

✅ No hay errores de metadatos para reprocesar.


In [9]:
# Cargar dataset con resúmenes
import pandas as pd
import modulos.paths as paths

df_discursos = pd.read_csv(paths.discursos_metadatos, encoding="utf-8-sig")

# Procesar discursos con el LLM
import modulos.prompts as prompts
from modulos.tipos_discurso import diccionario_tipos_discurso
from modulos.enunciacion import procesar_enunciacion_llm
from modulos.modelo import get_model_ollama

modelo_llm = get_model_ollama_par(modelo="gpt-oss:20b", temperature=0.0, output_format="text")

df_resultado = procesar_enunciacion_llm(
    df=df_discursos,
    modelo_llm=modelo_llm,
    diccionario=diccionario_tipos_discurso,
    prompt_enunciacion=prompts.PROMPT_ENUNCIACION,
    guardar=True,
    output_path=paths.discursos_enunc,
    mostrar_prompts=False,
    mostrar_tiempo=True,
    path_errores=paths.errores_enunciacion
)

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


✅ Archivo guardado: C:\PROYECTOS\EmoParse\data\A6. discursos_enunciacion.csv
⏱ Tiempo de procesar_enunciacion_llm: 41.44 s


In [10]:
# Reprocesar enunciación en caso de ser necesario

import pandas as pd
import modulos.paths as paths
from modulos.modelo import get_model_ollama_par
from modulos.reprocesamiento import reprocesar_enunciacion

path_errores = paths.errores_enunciacion
path_salida = paths.discursos_enunc

df_original = pd.read_csv(paths.discursos_enunc)

modelo_llm = get_model_ollama_par(modelo="gpt-oss:20b", temperature=0.0, output_format="text")

# Ejecutar reprocesamiento
df_corregido = reprocesar_enunciacion(
    df_original=df_original,
    path_errores=path_errores,
    modelo_llm=modelo_llm,
    intento=1,
    mostrar_prompts=False,
    path_salida=path_salida
)

✅ No hay errores de enunciación para reprocesar.


# 5. Prueba de módulo "identificacion_actores"

## 5.1. Identificación de actores

**Función:** identificar_actores_con_contexto

Esta función procesa una serie de recortes textuales para identificar actores relevantes mencionados en cada frase. Excluye automáticamente al enunciador y a los enunciatarios previamente detectados.  

Para ello, utiliza un modelo de lenguaje (LLM) a través de la API de Ollama y un **prompt estructurado** que combina:

- Información del discurso completo: resumen, fecha, lugar, tipo, enunciador y enunciatarios.  
- La frase objetivo y su contexto (frases anteriores y posteriores).  
- Heurísticas y ontología de actores posibles.

### Parámetros

- df_recortes (pd.DataFrame): DataFrame que contiene los recortes textuales a analizar. Debe incluir al menos las columnas:
  - 'frase'
  - 'recorte_id'
  - 'codigo' (identificador del discurso de inscripción)
- df_enunc (pd.DataFrame): DataFrame con los discursos completos y resultados de identificación enunciativa (enunciador, enunciatarios, tipo de discurso, lugar, etc.), asociados por la columna 'codigo'.
- prompt_actores (str, opcional): Prompt base con placeholders que será completado dinámicamente para cada recorte. Debe incluir:
  - {resumen_global}: Resumen del discurso de inscripción.
  - {fecha}: Fecha del discurso.
  - {lugar_justificacion}: Lugar del discurso (con justificación).
  - {tipo_discurso}: Tipo de discurso identificado.
  - {enunciador}: Enunciador del discurso.
  - {enunciatarios}: Enunciatarios identificados.
  - {frase}: Frase objetivo.
  - {frases_contexto}: Frases inmediatamente anteriores y posteriores a la frase objetivo.
  - {heuristicas}: Lista de reglas de inferencia válidas.
  - {ontologia}: Ontología de actores posibles (categorías permitidas para clasificación).
- path_errores (str, opcional): Ruta del archivo donde se guardarán los errores o casos que no puedan procesarse correctamente.
- output_path (str, opcional): Ruta del archivo donde se guardarán los resultados procesados en formato CSV.
- modelo_llm (callable, opcional): Función o clase del modelo LLM configurado. Debe recibir el prompt como string y devolver una respuesta.
- mostrar_prompts (bool, opcional): Si True, imprime el prompt generado para cada fragmento antes de enviarlo al modelo. Útil para debugging. Por defecto: False.
- guardar (bool, opcional): Si True, guarda el DataFrame resultante como CSV en output_path. Por defecto: True.
- checkpoint_interval (int, opcional): Frecuencia de guardado parcial en número de frases procesadas. Por defecto: 50.
- procesador (callable, opcional): Función encargada de procesar cada frase individual. Por defecto: procesar_una_frase.

### Retorna

- pd.DataFrame: Nuevo DataFrame con los actores identificados, con columnas como 'actor', 'tipo', 'modo', 'justificacion', y metadatos de la frase ('frase_idx', 'recorte_id', 'codigo') que permiten vincular cada actor con la frase correspondiente.

### Requiere

- Definición previa de los paths relevantes en el módulo modulos.paths.
- Prompt base definido en modulos.prompts.PROMPT_IDENTIFICAR_ACTORES.
- Preprocesamiento de enunciadores y enunciatarios mediante la función procesar_discursos_llm (u otra que genere df_enunc compatible).

In [1]:
import pandas as pd
import modulos.paths as paths

# Crear nuevo df de prueba
df_recortes = pd.read_csv(paths.recortes_preprocesado, encoding="utf-8-sig")
df_prueba = df_recortes.head(25)
df_prueba.to_csv("data/recortes_prueba.csv", index=False, encoding="utf-8")

In [2]:
import pandas as pd
import modulos.paths as paths
import modulos.prompts as prompts
from modulos.identificacion_actores import identificar_actores_con_contexto
from modulos.modelo import get_model_ollama_par

df_recortes = pd.read_csv(paths.recortes_preprocesado, encoding="utf-8-sig")
df_enunc = pd.read_csv(paths.discursos_enunc, encoding="utf-8-sig")

modelo_llm = get_model_ollama_par(modelo="gpt-oss:20b", temperature=0.0, output_format="text")

# Ejecutar identificación de actores
df_resultado = identificar_actores_con_contexto(
    df_recortes=df_prueba, # Reemplazar con df_recortes para análisis completo
    df_enunc=df_enunc,
    prompt_actores=prompts.PROMPT_IDENTIFICAR_ACTORES,
    path_errores=paths.errores_identificacion_actores,
    output_path=paths.actores_identificados,
    modelo_llm=modelo_llm,
    mostrar_prompts=False,
    guardar=True,
    mostrar_tiempo=True
)

C:\ProgramData\anaconda3\envs\ag_env2\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [06:16<00:00, 15.05s/it]

✅ Archivo guardado: C:\PROYECTOS\EmoParse\data\C1. actores_identificados.csv
⏱ Tiempo de identificar_actores_con_contexto: 376.30 s


In [3]:
df_actores = pd.read_csv(paths.actores_identificados, encoding="utf-8-sig")

df_actores.head(50)

,actor,tipo,modo,justificacion,frase_idx,recorte_id,codigo
0,audiencia presente,colectivo,inferido,El pronombre 'todos ustedes' se refiere a la a...,2,DISCURSO_001_FR_003,DISCURSO_001
1,Bettina Bulgheroni,humano_individual,explícito,Mencionada explícitamente como 'la flamante pr...,3,DISCURSO_001_FR_004,DISCURSO_001
2,autoridades de la institución,colectivo,explícito,Mencionadas explícitamente como 'todas las aut...,3,DISCURSO_001_FR_004,DISCURSO_001
3,presentes,colectivo,explícito,Mencionados explícitamente como 'todos los pre...,3,DISCURSO_001_FR_004,DISCURSO_001
4,grupo violento,colectivo,explícito,Mencionado explícitamente como 'un grupejo vio...,4,DISCURSO_001_FR_005,DISCURSO_001
5,nosotros,colectivo,inferido,Pronombre 'nosotros' indica un colectivo que e...,4,DISCURSO_001_FR_005,DISCURSO_001
6,la casta,colectivo,explícito,"referido explícitamente en la frase: ""la casta...",5,DISCURSO_001_FR_006,DISCURSO_001
7,los poderosos,colectivo,explícito,"referido explícitamente en la frase: ""los pode...",5,DISCURSO_001_FR_006,DISCURSO_001
8,el círculo rojo,colectivo,explícito,"referido explícitamente en la frase: ""junto al...",5,DISCURSO_001_FR_006,DISCURSO_001
9,la casta,colectivo,inferido,El verbo 'inventaron' aparece en tercera perso...,7,DISCURSO_001_FR_008,DISCURSO_001


## 5.2. Funciones de postprocesamiento de actores

### 5.2.1. Función para propagar actores por pronombres

Dado que no siempre el LLM lo realiza adecuadamente, esta función secundaria permite **propagar actores referidos por pronombres entre frases dentro de un mismo discurso**, asegurando mayor granularidad en la identificación de actores cuando las frases son cortas o abstractas.

```python
import pandas as pd
import modulos.paths as paths
from modulos.postprocesamiento_actores import propagar_actores_por_pronombres

# Cargar CSV con actores identificados previamente
df_actores = pd.read_csv(paths.actores_identificados)

# Aplicar propagación de pronombres
df_actores_con_pronombres = propagar_actores_por_pronombres(df_actores)

# Nota: esta operación no guarda el resultado automáticamente

### 5.2.2. Función para reprocesar errores de identificación de actores

En caso de que algunas frases no hayan sido procesadas correctamente por el LLM, esta función permite **reprocesar los errores registrados**, recuperando resultados potencialmente perdidos y guardando aquellos errores que persistan tras un nuevo intento.

In [3]:
import pandas as pd
import modulos.paths as paths
import modulos.prompts as prompts
from modulos.modelo import get_model_ollama_par
from modulos.reprocesamiento import reprocesar_errores_identificacion

# Cargar data
df_recortes = pd.read_csv(paths.recortes_preprocesado, encoding="utf-8-sig")
df_enunc = pd.read_csv(paths.discursos_enunc, encoding="utf-8-sig")

# Instanciar modelo
modelo_llm = get_model_ollama_par(modelo="gpt-oss:20b", temperature=0.0, output_format="text")

# Reprocesar errores con el mismo prompt utilizado originalmente
reprocesar_errores_identificacion(
    df_recortes=df_prueba,
    df_enunc=df_enunc,
    path_errores=paths.errores_identificacion_actores,
    path_salida=paths.actores_identificados,
    prompt_actores=prompts.PROMPT_IDENTIFICAR_ACTORES,
    intento=1,
    evitar_duplicados=True,
    modelo_llm=modelo_llm,
    mostrar_prompts=False
)

# Nota: la función guarda automáticamente los resultados recuperados y los errores persistentes en nuevos archivos
# Nota: Los errores de identificación pueden corresponder a frases donde el LLM no encuentra actores

🔁 Reprocesando 1 errores de identificación...


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


✅ Reprocesado INDEX=15
✅ CSV actualizado guardado en C:\PROYECTOS\EmoParse\data\C1. actores_identificados.csv
✅ Eliminados 1 errores exitosos del JSONL original


,actor,tipo,modo,justificacion,frase_idx,recorte_id,codigo
0,los que vienen,colectivo,inferido,Sujeto tácito / elíptico: el verbo 'envalenton...,15,DISCURSO_001_FR_016,DISCURSO_001


### 5.2.3. Función para validar actores según ontología

Esta función permite **validar actores previamente identificados usando un LLM**, comparando cada actor con los tipos definidos en la ontología. Los actores que cumplen los criterios se guardan en un archivo de válidos, mientras que los que no cumplen o generan respuestas ambiguas se guardan en un archivo de excluidos.

In [1]:
import pandas as pd
import modulos.paths as paths
from modulos.modelo import get_model_ollama_par
from modulos.postprocesamiento_actores import validacion_actores

path_df_actores = paths.actores_identificados
path_df_recortes = pd.read_csv("data/recortes_prueba.csv", encoding="utf-8-sig") # Para análisis completo, reemplazar el path con paths.recortes_preprocesado)
path_salida_validos = paths.actores_validos
path_salida_excluidos = paths.actores_excluidos

# Instanciar modelo
modelo_llm = get_model_ollama_par(modelo="gpt-oss:20b", temperature=0.0, output_format="text")

validacion_actores(
    path_df_actores=path_df_actores,
    path_df_recortes=path_df_recortes,
    path_salida_validos=path_salida_validos,
    path_salida_excluidos=path_salida_excluidos,
    modelo_llm=modelo_llm,
    mostrar_prompts=False
)

# Nota: la función recorre cada actor por recorte, llama al LLM con el prompt definido en `PROMPT_VALIDAR_ACTORES`,
# clasifica la respuesta como "Válido" o "Excluido" y guarda automáticamente los resultados en los archivos indicados.

C:\ProgramData\anaconda3\envs\ag_env2\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
Validando actores recorte DISCURSO_001_FR_003:   0%|                                             | 0/1 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"

Validando actores recorte DISCURSO_001_FR_004:   0%|                                             | 0/3 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"

Validando actores recorte DISCURSO_001_FR_004:  33%|████████████▎                        | 1/3 [00:01<00:02,  1.35s/it]INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"

Validando actores recorte DISCURSO_001_FR_004:  67%|████████████████████████▋            | 2/3 [00:03<00:01,  1.99s/it]INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTT

✅ Guardados 24 actores validados en 'C:\PROYECTOS\EmoParse\data\C2. actores_validos.csv'
📄 Guardados 1 actores excluidos en 'C:\PROYECTOS\EmoParse\data\C3. actores_excluidos.csv'


# 6. Prueba de módulo "deteccion_emociones"



In [1]:
# Prueba de función general: detecta emociones de enunciador, enunciatarios y actores en un solo prompt

import pandas as pd
import modulos.paths as paths
import modulos.prompts as prompts
from modulos.deteccion_emociones import identificar_emociones_con_contexto
from modulos.modelo import get_model_ollama_par
from modulos.schemas import ListaEmocionesSchema

# Cargar datos
df_recortes = pd.read_csv(paths.recortes_preprocesado, encoding="utf-8-sig")
df_discursos = pd.read_csv(paths.discursos_enunc, encoding="utf-8-sig")
df_actores = pd.read_csv(paths.actores_validos, encoding="utf-8-sig")  # Actores previamente identificados

# Crear nuevo df de prueba
df_prueba = df_recortes.head(5)
df_prueba.to_csv("data/recortes_prueba.csv", index=False, encoding="utf-8")

# Inicializar modelo
modelo_llm = get_model_ollama_par(modelo="gpt-oss:20b", temperature=0.0, output_format="text")

# Ejecutar identificación de emociones
df_emociones = identificar_emociones_con_contexto(
    df_recortes=df_prueba,
    df_discursos=df_discursos,
    df_actores=df_actores,
    schema=ListaEmocionesSchema,
    prompt_emociones=prompts.PROMPT_DETECCION_EMOCIONES,
    path_errores=paths.errores_identificacion_emociones,
    output_path=paths.emociones_identificadas,
    modelo_llm=modelo_llm,
    mostrar_prompts=False,
    guardar=True,
    mostrar_tiempo=True,
    checkpoint_interval=50,
    max_context=2
)

print(df_emociones)

C:\ProgramData\anaconda3\envs\ag_env2\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:16<00:00, 15.27s/it]

✅ Archivo guardado: C:\PROYECTOS\EmoParse\data\D1. emociones_identificadas.csv
⏱ Tiempo de identificar_emociones_con_contexto: 76.33 s
    experienciador    tipo_emocion modo_existencia  \
0       enunciador          placer       Realizada   
1       Enunciador       confianza       Realizada   
2     Javier Milei          placer       Realizada   
3     Javier Milei        gratitud       Realizada   
4     Javier Milei  agradecimiento       Realizada   
5     Javier Milei        humildad       Realizada   
6     Javier Milei        cortesía       Realizada   
7     Javier Milei           miedo       realizada   
8     Javier Milei     indignación       realizada   
9     Javier Milei        sorpresa       realizada   
10  grupo violento      hostilidad       realizada   

                                        justificacion  frase_idx  \
0   La frase contiene la expresión "es un placer",...          0   
1   El enunciador afirma que todavía conserva los ...          1   
2   La frase

In [2]:
# Prueba de función separada: detecta emociones de enunciador, enunciatarios y actores con prompts separados, aportando mayor granularidad y efectividad en la detección

import pandas as pd
import modulos.paths as paths
import modulos.prompts as prompts
from modulos.deteccion_emociones import identificar_emociones_todas
from modulos.modelo import get_model_ollama_par
from modulos.schemas import ListaEmocionesSchema
from modulos.tipos_discurso import diccionario_tipos_discurso

# Cargar datos
df_recortes = pd.read_csv(paths.recortes_preprocesado, encoding="utf-8-sig")
df_discursos = pd.read_csv(paths.discursos_enunc, encoding="utf-8-sig")
df_actores = pd.read_csv(paths.actores_validos, encoding="utf-8-sig")  # Actores previamente identificados

# Crear nuevo df de prueba
df_prueba = df_recortes.head(5)
df_prueba.to_csv("data/recortes_prueba.csv", index=False, encoding="utf-8-sig")

# Inicializar modelo
modelo_llm = get_model_ollama_par(modelo="gpt-oss:20b", temperature=0.0, output_format="text")

# Ejecutar identificación de emociones en tres niveles
resultados = identificar_emociones_todas(
    df_recortes=df_prueba,
    df_discursos=df_discursos,
    df_actores=df_actores,
    schema=ListaEmocionesSchema,
    modelo_llm=modelo_llm,
    prompt_enunciador=prompts.PROMPT_EMOCIONES_ENUNCIADOR,
    prompt_enunciatarios=prompts.PROMPT_EMOCIONES_ENUNCIATARIOS,
    prompt_actores=prompts.PROMPT_EMOCIONES_ACTORES,
    diccionario=diccionario_tipos_discurso,
    output_path=paths.emociones_identificadas,
    path_errores=paths.errores_identificacion_emociones,
    mostrar_prompts=False,
    guardar=True,
    mostrar_tiempo=True,
    checkpoint_interval=50,
    max_context=2
)

print(resultados)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:01<00:00, 12.33s/it]


✅ Archivo guardado: C:\PROYECTOS\EmoParse\data\D1. emociones_identificadas_enunciador.csv
⏱ Tiempo de identificar_emociones_con_contexto: 61.64 s


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:32<00:00, 18.56s/it]


✅ Archivo guardado: C:\PROYECTOS\EmoParse\data\D1. emociones_identificadas_enunciatarios.csv
⏱ Tiempo de identificar_emociones_con_contexto: 92.81 s


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:24<00:00, 16.81s/it]

✅ Archivo guardado: C:\PROYECTOS\EmoParse\data\D1. emociones_identificadas_actores.csv
⏱ Tiempo de identificar_emociones_con_contexto: 84.04 s
{'enunciador':    experienciador tipo_emocion modo_existencia  \
0    Javier Milei       placer       Realizada   
1    Javier Milei       placer       Realizada   
2    Javier Milei     gratitud       Realizada   
3    Javier Milei      orgullo       Realizada   
4    Javier Milei    confianza       Realizada   
5    Javier Milei       placer       Realizada   
6    Javier Milei     gratitud       Realizada   
7    Javier Milei     gratitud       Realizada   
8    Javier Milei  indignación       Realizada   
9    Javier Milei        miedo       Realizada   
10   Javier Milei          ira       Realizada   
11   Javier Milei  frustración       Realizada   
12   Javier Milei     sorpresa       Realizada   

                                        justificacion  frase_idx  \
0   El enunciador expresa explícitamente "es un pl...          0   
1   L